#Install libary


In [1]:
# !pip install black[jupyter]
!pip install -U tensorflow-addons
!pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Import and Initial Mount Disk

In [3]:
import os
# Mount drive
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/Colab Notebooks/face_recognize"
os.chdir(path)

import cv2
import io
import pickle
import math
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import csv
from PIL import Image
import torch
import torchvision.transforms as transforms
from train_tensorflow.inceptionresnetv1 import InceptionResNetV1
from train_tensorflow.models import call_instance_model, convert_model_to_embedding
from train_tensorflow.Classify import Classify
from tool.FormatFunction import FormatFunction
from tool.FileFunction import FileFunction
from tool.GlobalValue import GlobalValue

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Format Code

In [ ]:
# !black TensorFlow.ipynb

# Build train model

## Init value

In [ ]:
# global_value = GlobalValue(image_size=[80,80], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
# format_function = FormatFunction(global_value)

## Test Train Model tensorflow

In [ ]:
# # save model
# face_net_model.save("save_model/number_mnist_48")
# # save loss value
# loss_array = history.history['loss']
# with open("src/loss/mnist_loss.pkl", 'wb') as file:
#     pickle.dump(loss_array, file)

# reconstruct_model = tf.keras.models.load_model("save_model/modify_train_mnist")
# reconstruct_model = convert_train_model_to_embedding(reconstruct_model)
# results = reconstruct_model.predict(test_dataset)
# # Save test embeddings for visualization in projector
# np.savetxt("vecs_48.tsv", results, delimiter='\t')

# out_m = io.open('meta_48.tsv', 'w', encoding='utf-8')
# for img, labels in tfds.as_numpy(test_dataset):
#     [out_m.write(str(x) + "\n") for x in labels]
# out_m.close()


# try:
#   from google.colab import files
#   files.download('vecs_48.tsv')
#   files.download('meta_48.tsv')
# except:
#   pass

## Encoding Image

## Predict

In [4]:
global_value = GlobalValue(image_size=[160,160], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)
# Load network
path = os.path.join(os.getcwd(),"cache","data","label_dict.pkl")
with open(path, 'rb') as f:
  label_dict = pickle.load(f)
input_shape = (global_value.IMAGE_SIZE[0], global_value.IMAGE_SIZE[1], 3)
model = call_instance_model(input_shape,len(label_dict), 512,"Dense", "InceptionResNetV2")
model.load_weights(os.path.join(os.getcwd(),"save_model","160-64-InceptionResNetV2-Dense",f"epoch{18}.h5"))

model = convert_model_to_embedding(model)
# model.summary()
#Preprocess data
test_dataset = tf.data.Dataset.list_files("dataset/10_person/*/*",shuffle=False)
test_dataset = test_dataset.map(format_function.get_label_as_string, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(20)
# Evaluate the network
results = model.predict(test_dataset)

# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for imgs, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

127/127 [==============================] - 347s 3s/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# def load_pickle(path):
#     with open(path, 'rb') as f:
#         encoding_dict = pickle.load(f)
#     return encoding_dict

# recognition_t=5

# def detect(image, encoder,encoding_dict):
#     image = tf.expand_dims(image,0)
#     encode = encoder(image)
#     name = "unknow"
#     distance = float("inf")
#     for db_name, db_encode in encoding_dict.items():
#             dist = cosine(db_encode, encode)
#             if dist < recognition_t and dist < distance:
#                 name = db_name
#                 distance = dist

#     return name

# train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)
# train_dataset = train_dataset.map(format_function.format_gray_image)
# test_dataset = test_dataset.map(format_function.format_gray_image)
# combined_dataset = train_dataset.concatenate(test_dataset)
# combined_dataset = combined_dataset


# total = 0
# right_choice = 0
# i = 0
# total_image = len(combined_dataset)
# # plt.figure(figsize=(3*(total_image/10), 30))
# for image, label in combined_dataset:
#     predict_label = detect(image, face_net_model, load_pickle("encodings/encoding_train_cross_entropy.pkl"))
#     print("predict: {} |||||| actual: {}".format(predict_label, label))
#     # plt.subplot(total_image/10,10,i+1)
#     # plt.imshow(image.numpy())
#     # plt.title("predict: {} |||||| actual: {}".format(predict_label, label))
#     # plt.axis("off")
#     total = total + 1
#     if(tf.equal(predict_label,label)):
#         right_choice = right_choice + 1
#     i = i+1
# print(total, right_choice)

# Test embedding

# Evaluate

## Evaluate no mask lfw

In [ ]:
#Init
model_name = "110-ASIAN"
global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)
file_function = FileFunction()

for i in range(1,100):
  print("evaluate epoch: ", i)
  path_model = os.path.join(os.getcwd(),"save_model",model_name,"epoch{}".format(i))
  face_net_model = tf.keras.models.load_model(path_model)
  face_net_model = convert_train_model_to_embedding(face_net_model)
  classify = Classify(face_net_model, format_function)

  #Get embedding database
  encoding_path = os.path.join(os.getcwd(), "cache", "encodings", model_name, "epoch{}.pkl".format(i))
  if not os.path.exists(encoding_path):
    database_embedding = classify.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw_align"))
    classify.save_embedding_to_file(database_embedding, encoding_path)
  else:
    database_embedding = classify.load_embedding_from_file(encoding_path) 

  #Preprocess data
  data_directory = os.path.join(os.getcwd(), "dataset", "lfw_align")
  list_path = file_function.getPath(data_directory)

  # Accuracy
  precision, recall, accuracy,f1 = classify.evaluate_using_confusion_matrix(list_path, database_embedding, 100)
  with open(os.path.join(os.getcwd(),"cache","metrics",model_name+"(no mask).csv"), "a") as f:
    row = [i, precision, recall, accuracy, f1]
    writer = csv.writer(f)
    writer.writerow(row)
  print("precision {}, recall {}, accuracy {}, f1 {}".format(precision, recall, accuracy,f1))

## Evaluate mask lfw

In [ ]:
#Init
model_name = "110-ASIAN"
global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)
file_function = FileFunction()

for i in range(1,50):
  print("evaluate epoch: ", i)
  path_model = os.path.join(os.getcwd(),"save_model",model_name,"epoch{}".format(i))
  print(path_model)
  face_net_model = tf.keras.models.load_model(path_model)
  face_net_model = convert_train_model_to_embedding(face_net_model)
  classify = Classify(face_net_model, format_function)

  #Get embedding database
  encoding_path = os.path.join(os.getcwd(), "cache", "encodings", model_name, "epoch{}.pkl".format(i))
  if not os.path.exists(encoding_path):
    database_embedding = classify.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw_align"))
    classify.save_embedding_to_file(database_embedding, encoding_path)
  else:
    database_embedding = classify.load_embedding_from_file(encoding_path)

  #Preprocess data
  data_directory = os.path.join(os.getcwd(), "dataset", "lfw_mask")
  list_path = file_function.getPath(data_directory)
  
  # Accuracy
  precision, recall, accuracy,f1 = classify.evaluate_using_confusion_matrix(list_path, database_embedding, 100)
  with open(os.path.join(os.getcwd(),"cache","metrics",model_name+".csv"), "a") as f:
    row = [i, precision, recall, accuracy, f1]
    writer = csv.writer(f)
    writer.writerow(row)

## Compare time 128 vs 10575

In [ ]:
#Init
global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)
file_function = FileFunction()

for i in range(49,50):
  print("evaluate epoch: ", i)
  face_net_model_10575 = tf.keras.models.load_model("save_model/origin/face_recognize_entropy{}".format(i))
  face_net_model_128 = convert_train_model_to_embedding(face_net_model_10575)
  #Normalize output 
  outputs = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(face_net_model_10575.output)
  face_net_model_10575 = tf.keras.Model(face_net_model_10575.input, outputs)
  
  classify_10575 = Classify(face_net_model_10575, format_function)
  classify_128 = Classify(face_net_model_128, format_function)



  # Get embedding database
  database_embedding_10575 = classify_10575.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw_align"))
  database_embedding_128 = classify_128.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw_align"))
  
  # Preprocess data
  data_directory = os.path.join(os.getcwd(), "dataset", "lfw_align")
  list_path = file_function.getPath(data_directory)
  # Accuracy
  right_answer, unknow_answer, mis_answer, total_answer = classify_10575.evaluate(list_path, database_embedding_10575, 100)
  print("10575 accuracy {}".format(right_answer/total_answer))
  right_answer, unknow_answer, mis_answer, total_answer = classify_128.evaluate(list_path, database_embedding_128, 100)
  print("128 accuracy {}".format(right_answer/total_answer))

## Evaluate inception resnet entropy

# Save model weight to use locally

In [ ]:
face_net_model = tf.keras.models.load_model("save_model/face_recognize_entropy49")
face_net_model.save_weights("model49.h5")

In [ ]:
embedding = dict()
with open("database.pkl", 'wb') as file:
  pickle.dump(embedding, file)